Our goal is to explore models that predict NFL statistics using college game statistics. There is no one dataset that has both college and NFL career statistics for a set of players. We'll need to use multiple datasets from different sources in order to create a dataframe that works for our modelling.

First, we're going to need a dataframe containing information on players who played in both the NCAA and the NFL. We need to import separate data files for this step, so we'll make 2 different dataframes with data about college and NFL players then merge them together to figure out who exists in both places.

We'll also need a dataframe with both NFL game statistics and NCAA game statistics. Again, we'll need to use multiple datasets in order to make this, and we'll need to first make separate ones for NFL and NCAA stats. 
Once we have those two dataframes, we can merge them with the dataframe containing player information. We'll have one large dataframe with NFL and college statistics for players.

Before moving on, it's important to note that the data files with NFL information all have a player ID that stays constant throughout them, so there is a way to uniquely identify players and know whether they exist in each set of data. Similarly, the college datasets all have a Player Code for each player, and it's consistent throughout all the files so we can identify which datasets players exist in. These pieces of information will become important when we need to merge dataframes later on.

Our aim for this notebook is to grab the different datasets, clean and merge them appropriately, and have a final dataframe we can use to build our models (the code for which is in a different notebook).

Let's import the modules we'll be using throughout the remainder of the code.

In [28]:
import pandas as pd

We have our pandas module imported.

First, we need to read in all of the data files containing information about players. We'll read in the file with the NFL player data (nfl_players) as well as a file containing information about games that players participated in dring their NFL career (game participation). 

Then we can read in the college player data from 2008-2013. The datasets for college players have files for players in each individual season, so we need to read each season dataset in one at a time. We're also going to read in a file with information about college teams, and we'll need this further down when we start merging dataframes.

In [29]:
nfl_players = pd.read_csv('players_nfl.csv')
nfl_player_game_participation = pd.read_csv('gameParticipation.csv')
college_08 = pd.read_csv('player_2008.csv')
college_09 = pd.read_csv('player_2009.csv')
college_10 = pd.read_csv('player_2010.csv')
college_11 = pd.read_csv('player_2011.csv')
college_12 = pd.read_csv('player_2012.csv')
college_13 = pd.read_csv('player_2013.csv')
teams_college = pd.read_csv('team_2007.csv')

Ok, so now all the data files containing information about players have been read in as dataframes. 

Let's print out the first few rows of the nfl_player dataframe to see what we have for columns and info about it to see whether we have nan values we need to be concerned about.

In [30]:
nfl_players.head()

,playerId,nameFirst,nameLast,nameFull,position,collegeId,nflId,combineId,college,heightInches,...,dob,ageAtDraft,playerProfileUrl,homeCity,homeState,homeCountry,highSchool,hsCity,hsState,hsCountry
0,19770001,Ricky,Bell,Ricky Bell,RB,90089,2509366,NaN,USC,74.0,...,1955-04-08,22.084932,http://www.nfl.com/player/RickyBell/2509366/pr...,Houston,TX,USA,NaN,NaN,NaN,NaN
1,19770002,Tony,Dorsett,Tony Dorsett,RB,15260,2513084,NaN,Pittsburgh,71.0,...,1954-04-07,23.087671,http://www.nfl.com/player/TonyDorsett/2513084/...,Rochester,PA,USA,NaN,NaN,NaN,NaN
2,19770003,Eddie,Edwards,Eddie Edwards,DE,33124,2513479,NaN,Miami (FL),77.0,...,1954-04-25,23.038356,http://www.nfl.com/player/EddieEdwards/2513479...,Sumter,SC,USA,NaN,NaN,NaN,NaN
3,19770004,Marvin,Powell,Marvin Powell,OT,90089,2523462,NaN,USC,77.0,...,1955-08-30,21.690411,http://www.nfl.com/player/MarvinPowell/2523462...,Los Angeles,CA,USA,NaN,NaN,NaN,NaN
4,19770005,Gary,Jeter,Gary Jeter,DE,90089,2517530,NaN,USC,76.0,...,1955-01-24,22.287671,http://www.nfl.com/player/GaryJeter/2517530/pr...,Weirton,WV,USA,NaN,NaN,NaN,NaN


In [31]:
nfl_players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23295 entries, 0 to 23294
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   playerId          23295 non-null  int64  
 1   nameFirst         23295 non-null  object 
 2   nameLast          23295 non-null  object 
 3   nameFull          23295 non-null  object 
 4   position          23294 non-null  object 
 5   collegeId         23295 non-null  int64  
 6   nflId             20959 non-null  object 
 7   combineId         10097 non-null  float64
 8   college           23295 non-null  object 
 9   heightInches      20709 non-null  float64
 10  weight            20709 non-null  float64
 11  dob               20226 non-null  object 
 12  ageAtDraft        20226 non-null  float64
 13  playerProfileUrl  20959 non-null  object 
 14  homeCity          15618 non-null  object 
 15  homeState         15231 non-null  object 
 16  homeCountry       15715 non-null  object

The NFL player dataframe has rows representing individual players, and each player has a unique ID, which we're going to use later. The columns have logistical information about these players such as their height, weight, and position. There's also information about players' hometowns. The data types look to be appropriate to use later on.

We're going to now inspect the nfl_player_game_participation dataframe.

In [32]:
nfl_player_game_participation.head()

,gamePartId,gameId,teamId,playerId,gamePartUnit,gamePartSnapCount,nameFirst,nameLast,nameFull,position,...,dob,ageAtDraft,playerProfileUrl,homeCity,homeState,homeCountry,highSchool,hsCity,hsState,hsCountry
0,12100001,56473,3800,20140091,offense,39,John,Brown,John Brown,WR,...,1990-04-03,24.112329,http://www.nfl.com/player/JohnBrown/2543847/pr...,Homestead,FL,USA,Homestead HS,NaN,FL,USA
1,12100002,56473,3800,20100645,offense,33,John,Estes,John Estes,C,...,1987-03-25,23.093151,http://www.nfl.com/player/JohnEstes/2507981/pr...,Stockton,CA,USA,St. Mary's HS,Stockton,CA,USA
2,12100003,56473,3800,20120112,offense,30,Bobby,Massie,Bobby Massie,OT,...,1989-08-01,22.750685,http://www.nfl.com/player/BobbyMassie/2533543/...,Lynchburg,VA,USA,Hargrave Military Academy,Chatham,VA,USA
3,12100004,56473,3800,20100205,offense,30,Ted,Larsen,Ted Larsen,OG,...,1987-06-13,22.873973,http://www.nfl.com/player/TedLarsen/497026/pro...,Miami,FL,USA,Palm Harbor University HS,NaN,FL,USA
4,12100005,56473,3800,20090213,offense,30,Paul,Fanaika,Paul Fanaika,OG,...,1986-04-09,23.060274,http://www.nfl.com/player/PaulFanaika/71293/pr...,San Mateo,CA,USA,Mills HS,Millbrae,CA,USA


This dataframe has information about the games that NFL players participated in. Each row represents a player's participation in a game, which is identified by a game participation ID. The information about the players matches most of the data in the nfl_players dataframe.

We're going to need to know the number of games players played in for the modelling process. We can use the game participation dataframe to figure out the number of NFL games each player participated in. 

Let's figure out the number of games players participated in and put it into a dataframe. We can use map-reduce logic to make a dictionary that contains player ID and the number of games played in, and we'll map it onto the game participation dataframe. We'll make a new dataframe with just playerID and number of games played in and then use the playerID column to merge it with the nfl_players dataframe. We're going to print out the dataframe and make sure the column added correctly.

In [33]:
# select only the columns we need in the participation dataframe
nfl_game_total = nfl_player_game_participation[['playerId','gameId']] 

nfl_game_total = nfl_game_total.drop_duplicates()                                    # get rid of duplicates so we don't double count games
nfl_game_total = nfl_game_total.groupby('playerId').count()                          # group by playerId and count to get # of games
nfl_games = nfl_game_total.iloc[:,0].to_dict()                                       # make a dictionary with playerId and number of games
nfl_player_game_participation['Games played'] = nfl_player_game_participation['playerId'].map( nfl_games ) # add a column with number of games played by mapping the dictionary to it
numnfl_games = nfl_player_game_participation[['playerId', 'Games played']]            # dataframe with playerId and number of games played

nfl_players_and_games = pd.merge(numnfl_games, nfl_players, on= 'playerId')           # merge the dataframes on playerId
nfl_players_and_games = nfl_players_and_games.drop_duplicates()                       # get rid of duplicate players
nfl_players_and_games

,playerId,Games played,nameFirst,nameLast,nameFull,position,collegeId,nflId,combineId,college,...,dob,ageAtDraft,playerProfileUrl,homeCity,homeState,homeCountry,highSchool,hsCity,hsState,hsCountry
0,20140091,49,John,Brown,John Brown,WR,66762,2543847,18301.0,Pittsburg State (KS),...,1990-04-03,24.112329,http://www.nfl.com/player/JohnBrown/2543847/pr...,Homestead,FL,USA,Homestead HS,NaN,FL,USA
134,20100645,4,John,Estes,John Estes,C,96813,2507981,NaN,Hawaii,...,1987-03-25,23.093151,http://www.nfl.com/player/JohnEstes/2507981/pr...,Stockton,CA,USA,St. Mary's HS,Stockton,CA,USA
149,20120112,49,Bobby,Massie,Bobby Massie,OT,38677,2533543,17840.0,Mississippi,...,1989-08-01,22.750685,http://www.nfl.com/player/BobbyMassie/2533543/...,Lynchburg,VA,USA,Hargrave Military Academy,Chatham,VA,USA
424,20100205,42,Ted,Larsen,Ted Larsen,OG,27607,497026,17200.0,NC State,...,1987-06-13,22.873973,http://www.nfl.com/player/TedLarsen/497026/pro...,Miami,FL,USA,Palm Harbor University HS,NaN,FL,USA
632,20090213,18,Paul,Fanaika,Paul Fanaika,OG,85287,71293,16847.0,Arizona State,...,1986-04-09,23.060274,http://www.nfl.com/player/PaulFanaika/71293/pr...,San Mateo,CA,USA,Mills HS,Millbrae,CA,USA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458937,20160621,3,Taylor,Bertolet,Taylor Bertolet,K,77843,2556472,NaN,Texas A&M,...,1992-10-24,23.526027,http://www.nfl.com/player/TaylorBertolet/25564...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
458943,20191181,1,Sean,Smith,Sean Smith,P,45469,2563371,NaN,Dayton,...,NaN,NaN,http://www.nfl.com/player/SeanSmith/2563371/pr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
458945,20181249,2,Cameron,Nizialek,Cameron Nizialek,P,30602,2563375,NaN,Georgia,...,NaN,NaN,http://www.nfl.com/player/CameronNizialek/2563...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
458949,20191193,1,Marquis,Young,Marquis Young,RB,1003,2562928,NaN,Massachusetts,...,NaN,NaN,http://www.nfl.com/player/MarquisYoung/2562928...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Now we have a dataframe with information about NFL players along with a column that has the number of NFL games they played. We'll be able to use this for the model.

Now we'll concatenate the college dataframes so that we have all of the college player information in one dataframe.

The data covers several years, so it's likely to include the same players multiple times (a row for each of the years he played). We're going to drop duplicate values so that there's just one row for each player. We'll print the dataframe out at the end. 

In [34]:
player_data_08_13 = pd.concat([college_08, college_09])
player_data_08_13 = pd.concat([player_data_08_13, college_10])
player_data_08_13 = pd.concat([player_data_08_13, college_11])
player_data_08_13 = pd.concat([player_data_08_13, college_12])
player_data_08_13 = pd.concat([player_data_08_13, college_13])
player_college_data = player_data_08_13.drop_duplicates()          # drop duplicate values so we don't have the same player twice
player_college_data

,Player Code,Team Code,Last Name,First Name,Uniform Number,Class,Position,Height,Weight,Home Town,Home State,Home Country,Last School
0,85455,5,Allen,Alex,10,JR,RB,72.0,205.0,Youngstown,OH,US,Ursuline HS
1,85457,5,Anderson,Brandon,1,SR,DB,71.0,175.0,Dublin,VA,US,Pulaski HS/Hargrave Military Academy
2,1007928,5,Anderson,Jake,73,FR,OL,77.0,305.0,Erie,PA,US,McDowell HS
3,85458,5,Anderson,Zack,78,JR,OL,76.0,315.0,Lester,PA,US,Interboro HS
4,81392,5,Bain,Ryan,92,JR,DL,74.0,300.0,Bolingbrook,IL,US,Bolingbrook HS/Univ. of Iowa
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21787,1059626,2915,Williams,Jamario,20,FR,CB,NaN,NaN,NaN,NaN,NaN,NaN
21789,1059658,2915,Windham,Hunter,NaN,FR,RB,NaN,NaN,NaN,NaN,NaN,NaN
21790,1044983,2915,Woodson,Brodie,46,JR,LB,NaN,NaN,NaN,NaN,NaN,NaN
21791,1059632,2915,Young,Dylan,45,FR,LB,NaN,NaN,NaN,NaN,NaN,NaN


Ok, so we now have one dataframe with information about college players from 2008-2013. The rows represent a player, each of which has the Player Code mentioned earlier, and the columns contain logistical personal information about the players and information about his class year, position, and uniform number in college.

We're going to merge this dataframe with the dataframe containing information about teams. This will give us the team name in addition to the team code. Preliminary data exploration told us that both dataframes have a consistent Team Code column, so we can merge on that value.

In [35]:
team_and_player = pd.merge(player_college_data, teams_college, on='Team Code')
team_and_player

,Player Code,Team Code,Last Name,First Name,Uniform Number,Class,Position,Height,Weight,Home Town,Home State,Home Country,Last School,Name,Conference Code
0,85455,5,Allen,Alex,10,JR,RB,72.0,205.0,Youngstown,OH,US,Ursuline HS,Akron,875
1,85457,5,Anderson,Brandon,1,SR,DB,71.0,175.0,Dublin,VA,US,Pulaski HS/Hargrave Military Academy,Akron,875
2,1007928,5,Anderson,Jake,73,FR,OL,77.0,305.0,Erie,PA,US,McDowell HS,Akron,875
3,85458,5,Anderson,Zack,78,JR,OL,76.0,315.0,Lester,PA,US,Interboro HS,Akron,875
4,81392,5,Bain,Ryan,92,JR,DL,74.0,300.0,Bolingbrook,IL,US,Bolingbrook HS/Univ. of Iowa,Akron,875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116160,1050889,748,Williams,Jarrid,4,SO,LB,NaN,NaN,NaN,NaN,NaN,NaN,Wagner,846
116161,1060964,748,Williams,Lawrence,91,SR,DL,NaN,NaN,NaN,NaN,NaN,NaN,Wagner,846
116162,1050884,748,Williams,Nevon,12,JR,DB,NaN,NaN,NaN,NaN,NaN,NaN,Wagner,846
116163,1050902,748,Wytanis,Ryan,61,SO,OL,NaN,NaN,NaN,NaN,NaN,NaN,Wagner,846


The college player dataframe (called team_and_player now) has the college name now too, which is represented by the column called "Name".

The Name column should be renamed so that we know it's the name of the college and not the player's name. We also don't need all of these columns, so let's keep just the ones we'll need later and then print the dataframe.

In [36]:
team_and_player.rename(columns = {'Name':'College Name'}, inplace = True)      # rename the column to be College Name
team_and_player = team_and_player[['Player Code', 'Team Code', 'Last Name', 'First Name', 'Class', 'Position', 'Home Town', 'Home State', 'College Name', 'Conference Code']]
team_and_player.head()

,Player Code,Team Code,Last Name,First Name,Class,Position,Home Town,Home State,College Name,Conference Code
0,85455,5,Allen,Alex,JR,RB,Youngstown,OH,Akron,875
1,85457,5,Anderson,Brandon,SR,DB,Dublin,VA,Akron,875
2,1007928,5,Anderson,Jake,FR,OL,Erie,PA,Akron,875
3,85458,5,Anderson,Zack,JR,OL,Lester,PA,Akron,875
4,81392,5,Bain,Ryan,JR,DL,Bolingbrook,IL,Akron,875


The dataframe has a column called "College Name" now and only the column's we're going to need for the rest of the code.

Now we're ready to merge the NFL player dataframe with the college player dataframe. 

Based on preliminary data exploration, there are players with the same name who played on the same college team, played the same position, and are from the same state. So we're going to merge on player name, position, college name, home city, and home state.

After we merge them we'll keep only the columns we need and get rid of the rest. We'll print out the first 5 rows of the dataframe.

In [37]:
merged_college_nfl = pd.merge(nfl_players_and_games, team_and_player, left_on=['nameFirst', 'nameLast', 'position', 'college', 'homeCity', 'homeState'], right_on=['First Name', 'Last Name', 'Position', 'College Name', 'Home Town', 'Home State'])

# keep just the columns we need
merged_college_nfl = merged_college_nfl[['playerId', 'nameFirst', 'nameLast', 'nameFull', 'position', 'collegeId', 'college', 'Games played', 'nflId', 'Player Code', 'Team Code', 'Position', 'College Name', 'Conference Code']]

# print first 5 rows
merged_college_nfl.head()

,playerId,nameFirst,nameLast,nameFull,position,collegeId,college,Games played,nflId,Player Code,Team Code,Position,College Name,Conference Code
0,20120112,Bobby,Massie,Bobby Massie,OT,38677,Mississippi,49,2533543,1024974,433,OT,Mississippi,911
1,20120591,Bradley,Sowell,Bradley Sowell,OT,38677,Mississippi,51,2535866,1010000,433,OT,Mississippi,911
2,20130187,Andre,Ellington,Andre Ellington,RB,29631,Clemson,35,2539217,1015945,147,RB,Clemson,821
3,20130187,Andre,Ellington,Andre Ellington,RB,29631,Clemson,35,2539217,1015945,147,RB,Clemson,821
4,20130187,Andre,Ellington,Andre Ellington,RB,29631,Clemson,35,2539217,1015945,147,RB,Clemson,821


We know that we have all the columns we need, and there are duplicate players. But each row represents a player who appeared in both the NFL and NCAA player datasets, so we have a group of players we can find both NFL and college statistics for.

Let's print out information about the dataframe to make sure that there are no nan values.

In [38]:
merged_college_nfl.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1668 entries, 0 to 1667
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   playerId         1668 non-null   int64 
 1   nameFirst        1668 non-null   object
 2   nameLast         1668 non-null   object
 3   nameFull         1668 non-null   object
 4   position         1668 non-null   object
 5   collegeId        1668 non-null   int64 
 6   college          1668 non-null   object
 7   Games played     1668 non-null   int64 
 8   nflId            1668 non-null   object
 9   Player Code      1668 non-null   int64 
 10  Team Code        1668 non-null   int64 
 11  Position         1668 non-null   object
 12  College Name     1668 non-null   object
 13  Conference Code  1668 non-null   int64 
dtypes: int64(6), object(8)
memory usage: 195.5+ KB


There aren't any nan values in the dataframe, so we know that each player was in both the NFL and NCAA dataframes. Note that their playerIds (NFL) and Player Codes (NCAA) are included to use for the model making.

Some of the rows are duplicates, so let's drop duplicate rows. We're also going to drop any nans just to be safe. We'll print out information about the dataframe.

In [39]:
merged_college_nfl = merged_college_nfl.dropna(axis=0)
merged_college_nfl = merged_college_nfl.drop_duplicates()
merged_college_nfl.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 708 entries, 0 to 1667
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   playerId         708 non-null    int64 
 1   nameFirst        708 non-null    object
 2   nameLast         708 non-null    object
 3   nameFull         708 non-null    object
 4   position         708 non-null    object
 5   collegeId        708 non-null    int64 
 6   college          708 non-null    object
 7   Games played     708 non-null    int64 
 8   nflId            708 non-null    object
 9   Player Code      708 non-null    int64 
 10  Team Code        708 non-null    int64 
 11  Position         708 non-null    object
 12  College Name     708 non-null    object
 13  Conference Code  708 non-null    int64 
dtypes: int64(6), object(8)
memory usage: 83.0+ KB


Now there's just one row for each of the individual players, and none of the columns have nan values. Note the playerId and Player Code columns we'll need to get game statistics.

We have the player information dataframe that we need, so we can move on to getting the NFL and college statistics for the players in our dataframe.

We'll first read in the files with the college statistics and save them to dataframes. Again, we're only looking at statistics for 2008-2013 seasons.

In [40]:
college_stats08 = pd.read_csv('player-game-statistics 08.csv')
college_stats09 = pd.read_csv('player-game-statistics 09.csv')
college_stats10 = pd.read_csv('player-game-statistics 10.csv')
college_stats11 = pd.read_csv('player-game-statistics 11.csv')
college_stats12 = pd.read_csv('player-game-statistics 12.csv')
college_stats13 = pd.read_csv('player-game-statistics 13.csv')

We've read in all the dataframes, and prior exploration tells us that each one has the same columns.

Let's concatenate all the college statistics dataframes so that we have one dataframe with all of the NCAA stats. We'll print out the dataframe to get an idea of the columns we have and to make sure that they lined up correctly.

In [41]:
all_college_stats = pd.concat([college_stats08, college_stats09])
all_college_stats = pd.concat([all_college_stats, college_stats10])
all_college_stats = pd.concat([all_college_stats, college_stats11])
all_college_stats = pd.concat([all_college_stats, college_stats12])
all_college_stats = pd.concat([all_college_stats, college_stats13])

all_college_stats

,Player Code,Game Code,Rush Att,Rush Yard,Rush TD,Pass Att,Pass Comp,Pass Yard,Pass TD,Pass Int,...,Tackle Solo,Tackle Assist,Tackle For Loss,Tackle For Loss Yard,Sack,Sack Yard,QB Hurry,Fumble Forced,Pass Broken Up,Kick/Punt Blocked
0,85455,5079620080830,4,13,0,0,0,0,0,0,...,0,0,0.0,0,0.0,0,0,0,0,0
1,85455,5068820080906,21,103,2,0,0,0,0,0,...,0,0,0.0,0,0.0,0,0,0,0,0
2,85455,47000520080913,7,36,1,0,0,0,0,0,...,0,0,0.0,0,0.0,0,0,0,0,0
3,85457,5079620080830,0,0,0,0,0,0,0,0,...,3,1,0.0,0,0.0,0,0,0,0,0
4,85457,5068820080906,0,0,0,0,0,0,0,0,...,0,3,0.0,0,0.0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60578,1059641,2915005120130831,0,0,0,0,0,0,0,0,...,0,0,0.0,0,0.0,0,0,0,0,0
60579,1054097,2915005120130831,0,0,0,0,0,0,0,0,...,0,0,0.0,0,0.0,0,0,0,0,0
60580,1044972,2915005120130831,4,6,0,6,1,6,0,1,...,0,0,0.0,0,0.0,0,0,0,0,0
60581,1044986,2915005120130831,0,0,0,0,0,0,0,0,...,2,0,0.0,0,0.0,0,0,1,0,0


The all_college_stats dataframe was created successfuly, and each row represents a player's statistics for a game. It has the Player Code column that matches the Player Code in the player information dataframe. There are more than 50 columns containing data about statistics from games, and they all have the appropriate data type.

We're not going to need all of these statistics, so let's filter for just the columns that will be useful for making a model.

In [68]:
# keep columns we'll need
all_college_stats = all_college_stats[['Player Code', 'Game Code', 'Rush Att', 'Rush Yard', 'Rush TD', 'Pass Att', 'Pass Comp', 'Pass Yard', 'Pass TD', 'Rec', 'Rec Yards', 'Rec TD', 'Fumble', 'Fumble Lost', 'Tackle Solo', 'Tackle Assist', 'Tackle For Loss', 'Tackle For Loss Yard', 'Sack', 'Sack Yard', 'Fumble Forced', 'Pass Broken Up']]

# sort by player code so we can see if there's duplicates
all_college_stats.sort_values(by='Player Code')

# print dataframe
all_college_stats

,Player Code,Game Code,Rush Att,Rush Yard,Rush TD,Pass Att,Pass Comp,Pass Yard,Pass TD,Rec,...,Fumble,Fumble Lost,Tackle Solo,Tackle Assist,Tackle For Loss,Tackle For Loss Yard,Sack,Sack Yard,Fumble Forced,Pass Broken Up
0,85455,5079620080830,4,13,0,0,0,0,0,2,...,0,0,0,0,0.0,0,0.0,0,0,0
1,85455,5068820080906,21,103,2,0,0,0,0,0,...,0,0,0,0,0.0,0,0.0,0,0,0
2,85455,47000520080913,7,36,1,0,0,0,0,0,...,0,0,0,0,0.0,0,0.0,0,0,0
3,85457,5079620080830,0,0,0,0,0,0,0,0,...,0,0,3,1,0.0,0,0.0,0,0,0
4,85457,5068820080906,0,0,0,0,0,0,0,0,...,0,0,0,3,0.0,0,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60578,1059641,2915005120130831,0,0,0,0,0,0,0,0,...,0,0,0,0,0.0,0,0.0,0,0,0
60579,1054097,2915005120130831,0,0,0,0,0,0,0,0,...,0,0,0,0,0.0,0,0.0,0,0,0
60580,1044972,2915005120130831,4,6,0,6,1,6,0,0,...,1,1,0,0,0.0,0,0.0,0,0,0
60581,1044986,2915005120130831,0,0,0,0,0,0,0,0,...,0,0,2,0,0.0,0,0.0,0,1,0


The college statistics dataframe now has only columns with relevant statistics for the model we'll make later on. 

Let's figure out how many games each college player participated in like we did for NFL players and NFL games. This will be useful when we go through the modelling process.

We'll use a similar technique that we used with the NFL players and games and map a dictionary with player code and games played to the dataframe.

In [70]:
# extract just the player and game codes
players_and_games = all_college_stats[['Player Code','Game Code']] 

# drop duplicates so the same game doesn't appear more than once
players_and_games = players_and_games.drop_duplicates()

# group by player code and count # of time it appears
players_and_games = players_and_games.groupby('Player Code').count()

# convert it to a dictionary and map it to a new column for games played in college stats df
num_games_for_player = players_and_games.iloc[:,0].to_dict()
all_college_stats['NCAA Games played'] = all_college_stats['Player Code'].map( num_games_for_player )

# make df with just player code and games played
num_games = all_college_stats[['Player Code', 'NCAA Games played']]
num_games

,Player Code,NCAA Games played
0,85455,27
1,85455,27
2,85455,27
3,85457,11
4,85457,11
...,...,...
60578,1059641,1
60579,1054097,1
60580,1044972,2
60581,1044986,1


Now we have a dataframe (num_games) with player code and the number of NCAA games each player participated in.

There's duplicate values, so we can drop them now to ensure each player has just one row. We'll print out the dataframe to ensure that we drop them successfully.

In [44]:
num_games = num_games.drop_duplicates()
num_games

,Player Code,NCAA Games played
0,85455,27
3,85457,11
14,85458,3
16,85459,9
24,85461,2
...,...,...
60576,1036772,1
60577,1054110,1
60578,1059641,1
60579,1054097,1


Now the num_games dataframe has Player Code and the number of college games the player played in.

Let's take our college stats dataframe and make a dataframe with totals for the players on each type of statistic. This will give us total statistics for all games that the players played in.

Once we've done that, we can merge that new dataframe with the num_games dataframe using the Player Code column. We'll drop all the duplicate rows. Then we'll have a dataframe with total statistics and total NCAA games played for each player.

In [76]:
# group by player code and sum up all columns with stats
new_college_stats = all_college_stats.groupby('Player Code')[['Rush Att', 'Rush Yard', 'Rush TD', 'Pass Att', 'Pass Comp', 'Pass Yard', 'Pass TD', 'Rec', 'Rec Yards', 'Rec TD', 'Fumble', 'Tackle Solo', 'Tackle Assist', 'Tackle For Loss', 'Tackle For Loss Yard', 'Sack', 'Sack Yard', 'Fumble Forced', 'Pass Broken Up']].sum()
# reset index to start from 0
new_college_stats.reset_index()

# merge df with stats with the number of games player
new_college_stats = pd.merge(new_college_stats, num_games, on = 'Player Code')
# drop duplicate values
new_college_stats = new_college_stats.drop_duplicates()
new_college_stats

,Player Code,Rush Att,Rush Yard,Rush TD,Pass Att,Pass Comp,Pass Yard,Pass TD,Rec,Rec Yards,...,Fumble,Tackle Solo,Tackle Assist,Tackle For Loss,Tackle For Loss Yard,Sack,Sack Yard,Fumble Forced,Pass Broken Up,NCAA Games played
0,4,0,0,0,0,0,0,0,26,313,...,0,1,0,0.0,0,0.0,0,0,0,10
10,6,0,0,0,0,0,0,0,1,23,...,2,0,0,0.0,0,0.0,0,0,0,6
16,10,0,0,0,0,0,0,0,0,0,...,0,0,0,0.0,0,0.0,0,0,0,4
20,15,0,0,0,0,0,0,0,0,0,...,0,0,0,0.0,0,0.0,0,0,0,1
21,18,1,-10,0,0,0,0,0,0,0,...,1,1,0,0.0,0,0.0,0,0,0,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347119,1061676,0,0,0,0,0,0,0,0,0,...,0,1,0,0.0,0,0.0,0,0,0,1
347120,1061680,0,0,0,0,0,0,0,0,0,...,0,3,0,1.0,1,0.0,0,0,1,1
347121,1061684,0,0,0,0,0,0,0,0,0,...,0,0,1,0.0,0,0.0,0,0,0,1
347122,1061685,0,0,0,0,0,0,0,0,0,...,0,1,2,0.0,0,0.0,0,0,0,1


Now we have a dataframe that contains one row for each player, and there's total statistics for all games he played in college and the total number of games he played in.

When we make the models we'll want to use per game and per play variables, so let's create the variables we want to use now.

We can divide different columns to get these statistics on a per play and per game basis.

In [77]:
new_college_stats['NCAA Yards per Carry'] = new_college_stats['Rush Yard']/new_college_stats['Rush Att']
new_college_stats['NCAA Rush TDs per Game'] = new_college_stats['Rush TD']/new_college_stats['NCAA Games played']
new_college_stats['NCAA Pass Completion'] = (new_college_stats['Pass Comp']/new_college_stats['Pass Att'])/new_college_stats['NCAA Games played']
new_college_stats['NCAA Yards per Completion'] = new_college_stats['Pass Yard']/new_college_stats['Pass Comp']
new_college_stats['NCAA Pass TDs per Game'] = new_college_stats['Pass TD']/new_college_stats['NCAA Games played']
new_college_stats['NCAA Reception TDs per Game'] = new_college_stats['Rec TD']/new_college_stats['NCAA Games played']
new_college_stats['NCAA Yards per Reception'] = new_college_stats['Rec Yards']/new_college_stats['Rec']
new_college_stats['NCAA Solo Tackle per Game'] = new_college_stats['Tackle Solo']/new_college_stats['NCAA Games played']
new_college_stats['NCAA Assisted Tackle per Game'] = new_college_stats['Tackle Assist']/new_college_stats['NCAA Games played']
new_college_stats['NCAA Tackle for Loss per Game'] = new_college_stats['Tackle For Loss']/new_college_stats['NCAA Games played']
new_college_stats['NCAA Yards per Tackle for Loss'] = new_college_stats['Tackle For Loss Yard']/new_college_stats['Tackle For Loss']
new_college_stats['NCAA Sacks per Game'] = new_college_stats['Sack']/new_college_stats['NCAA Games played']
new_college_stats['NCAA Yards per Sack'] = new_college_stats['Sack Yard']/new_college_stats['Sack']
new_college_stats['NCAA Forced Fumbles per Game'] = new_college_stats['Fumble Forced']/new_college_stats['NCAA Games played']
new_college_stats['NCAA Fumbles per Game'] = new_college_stats['Fumble']/new_college_stats['NCAA Games played']
new_college_stats['NCAA Broken up Pass per Game'] = new_college_stats['Pass Broken Up']/new_college_stats['NCAA Games played']

Ok, so now we've created new columns to put statistics on a per play and per game basis. Without displaying the dataframe, we know that there are more columns than we actually need. 

Let's keep only the statistics that we have on a per game and per play basis (with the exception of the pass completion statistic). We're also going to round the statistics to 2 decimal places to have more compact statistics. Then we can print out the dataframe.

In [78]:
new_college_stats = new_college_stats[['Player Code', 'NCAA Games played','NCAA Yards per Carry','NCAA Rush TDs per Game', 'NCAA Pass Completion', 'NCAA Yards per Completion', 'NCAA Pass TDs per Game', 'NCAA Reception TDs per Game', 'NCAA Yards per Reception', 'NCAA Solo Tackle per Game', 'NCAA Assisted Tackle per Game',  'NCAA Tackle for Loss per Game','NCAA Yards per Tackle for Loss', 'NCAA Sacks per Game', 'NCAA Yards per Sack', 'NCAA Forced Fumbles per Game', 'NCAA Fumbles per Game', 'NCAA Broken up Pass per Game'  ]]

# set variable for rounding stats to 2 decimal places
decimals = 2    

# round columns to 2 decimals 
new_college_stats = new_college_stats[['Player Code', 'NCAA Games played','NCAA Yards per Carry','NCAA Rush TDs per Game', 'NCAA Pass Completion', 'NCAA Yards per Completion', 'NCAA Pass TDs per Game', 'NCAA Reception TDs per Game', 'NCAA Yards per Reception', 'NCAA Solo Tackle per Game', 'NCAA Assisted Tackle per Game',  'NCAA Tackle for Loss per Game','NCAA Yards per Tackle for Loss', 'NCAA Sacks per Game', 'NCAA Yards per Sack', 'NCAA Forced Fumbles per Game', 'NCAA Fumbles per Game', 'NCAA Broken up Pass per Game']].apply(lambda x: round(x, decimals))
new_college_stats

,Player Code,NCAA Games played,NCAA Yards per Carry,NCAA Rush TDs per Game,NCAA Pass Completion,NCAA Yards per Completion,NCAA Pass TDs per Game,NCAA Reception TDs per Game,NCAA Yards per Reception,NCAA Solo Tackle per Game,NCAA Assisted Tackle per Game,NCAA Tackle for Loss per Game,NCAA Yards per Tackle for Loss,NCAA Sacks per Game,NCAA Yards per Sack,NCAA Forced Fumbles per Game,NCAA Fumbles per Game,NCAA Broken up Pass per Game
0,4,10,NaN,0.0,NaN,NaN,0.0,0.3,12.04,0.10,0.0,0.0,NaN,0.0,NaN,0.0,0.00,0.0
10,6,6,NaN,0.0,NaN,NaN,0.0,0.0,23.00,0.00,0.0,0.0,NaN,0.0,NaN,0.0,0.33,0.0
16,10,4,NaN,0.0,NaN,NaN,0.0,0.0,NaN,0.00,0.0,0.0,NaN,0.0,NaN,0.0,0.00,0.0
20,15,1,NaN,0.0,NaN,NaN,0.0,0.0,NaN,0.00,0.0,0.0,NaN,0.0,NaN,0.0,0.00,0.0
21,18,13,-10.0,0.0,NaN,NaN,0.0,0.0,NaN,0.08,0.0,0.0,NaN,0.0,NaN,0.0,0.08,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347119,1061676,1,NaN,0.0,NaN,NaN,0.0,0.0,NaN,1.00,0.0,0.0,NaN,0.0,NaN,0.0,0.00,0.0
347120,1061680,1,NaN,0.0,NaN,NaN,0.0,0.0,NaN,3.00,0.0,1.0,1.0,0.0,NaN,0.0,0.00,1.0
347121,1061684,1,NaN,0.0,NaN,NaN,0.0,0.0,NaN,0.00,1.0,0.0,NaN,0.0,NaN,0.0,0.00,0.0
347122,1061685,1,NaN,0.0,NaN,NaN,0.0,0.0,NaN,1.00,2.0,0.0,NaN,0.0,NaN,0.0,0.00,0.0


Now we have a dataframe with all college statistics. The rows represent players' cumulative stats across their college career on a per play and per game basis.

Let's read in the NFL statistics data files and save them to their own dataframes. Each data file contains statistics for different types of plays, and preliminary exploration of them gave us an idea of what columns they have, but we'll print just the fumbles df as an example.

In [48]:
nfl_fumbles = pd.read_csv('fumbles.csv')
nfl_interceptions = pd.read_csv('interceptions.csv')
nfl_defend_pass = pd.read_csv('passDef.csv')
nfl_passer = pd.read_csv('passer.csv')
nfl_receiver = pd.read_csv('receiver.csv')
nfl_rusher = pd.read_csv('rusher.csv')
nfl_sacks = pd.read_csv('sacks.csv')
nfl_tackles = pd.read_csv('tackles.csv')

In [80]:
nfl_fumbles

,fumId,playId,teamId,playerId,fumPosition,fumType,fumOOB,fumTurnover,fumNull
0,800001,1,3800,20020185,RB,1,0,0.0,0
1,800002,27,3200,20000239,FB,1,0,0.0,0
2,800003,53,3200,20000199,QB,0,0,1.0,0
3,800004,56,3800,20020081,QB,1,0,0.0,0
4,800005,84,3800,19970057,LB,1,1,0.0,0
...,...,...,...,...,...,...,...,...,...
14905,15242571,15084212,4600,20150069,WR,1,1,0.0,0
14906,15242572,15084271,4900,20180038,RB,1,0,1.0,0
14907,15242573,15084330,200,20080003,QB,1,0,1.0,0
14908,15242574,15084351,4900,20170581,RB,1,0,0.0,0


Now we have all of the NFL statistics read in as dataframes. Based on the output of the fumbles df, each row represents a play in a game and identifies which player was involved in the play for the relevant statistic. The column details the statistics for the specific aspects of the play.  

They all have columns we aren't going to need to make models, so let's cut down on the columns.

Make a function that allows us to keep only the columns we care about in each of the nfl_stats dataframes. We also want the dataframe to sum up statistics for the players so we have totals rather than statistics for every play the player was involved in.

In [79]:
def make_df_for_nfl_stat(df, *args):                                 # take name of df and columns we want to keep
    nfl_total_stat = df.groupby('playerId')[[*args]].sum()           # group on player id and sum up total stats for columns we pass
    nfl_total_stat = nfl_total_stat.reset_index()                    # reset index 
    nfl_total_stat = pd.DataFrame(nfl_total_stat)                    # make a df out of the total stats
    return nfl_total_stat                                            # return the df

Now we have a function that will return a dataframe with just the player ID and the relevant columns for each type of play/statistic in the dataframe that's passed in as an argument. The columns will represent sums of statistics in all games for players.

Let's use this function to make new dataframes for each of the NFL statistics/plays dataframes. Again, based on preliminary exploration of these dataframes, we already know what columns we want to keep.

We'll start with interception. We want to keep just the int and intYards columns. The int column tells us how many interceptions a player had on a play, and the intYards tells us how many yards they ran the ball for after that interception.

In [51]:
nfl_total_interceptions_yards = make_df_for_nfl_stat(nfl_interceptions, 'int', 'intYards')
nfl_total_interceptions_yards

,playerId,int,intYards
0,19890005,5,144
1,19900005,3,28
2,19910031,1,26
3,19920005,3,30
4,19920007,5,86
...,...,...,...
1487,20190874,2,38
1488,20190880,2,118
1489,20190954,2,30
1490,20190990,2,48


Now we have a dataframe dedicated to the interception statistic. The dataframe tells us the total number of interceptions the player had and the total number of yards they've run the ball for after interceptions.

We'll do the same for the forced fumbles dataframe. Prior inspection tells us that the Fumble Type column has values of forced and unforced, but we want just forced. We'll change the values to be 1 for forced and 0 for unforced so we can sum up the 1 values and have total number of forced fumbles. It makes sense to change the name of this column to be NFL Forced Fumbles rather than fumble type. We're going to keep the fumTurnover column too, which tells us whether the fumble on the play resulted in a turnover.

In [52]:
# changed forced and unforced to values of 0 and 1
nfl_fumbles = nfl_fumbles.replace(to_replace='forced', value=1)
nfl_fumbles = nfl_fumbles.replace(to_replace='unforced', value=0)

# make the new dataframe for the columns we want
nfl_total_forced_fumbles = make_df_for_nfl_stat(nfl_fumbles, 'fumType', 'fumTurnover')

# change name of column to be forced fumbles and print it out
nfl_total_forced_fumbles = nfl_total_forced_fumbles.rename({'fumType': 'NFL Forced Fumbles'}, axis=1)
nfl_total_forced_fumbles

,playerId,NFL Forced Fumbles,fumTurnover
0,19850285,2,2.0
1,19870001,13,12.0
2,19870098,3,1.0
3,19880006,2,1.0
4,19880076,1,1.0
...,...,...,...
2692,20191023,1,1.0
2693,20191032,0,1.0
2694,20191036,1,1.0
2695,20191044,0,0.0


The dataframe above tells us the total number of forced fumbles players have caused in all games they've played in and the number of those fumbles that have resulted in a turnover.

Let's make a dataframe for the passer dataframe. First, we want to make sure we have the passLength columns as it tells us how many yards the player threw for on the pass. We'll keep the pass attempt and complete columns so we can calculate the passer's completion percentage. We'll also keep the passTD and passInt columns so we know how many passes players have thrown that resulted in touchdowns and interceptions.

After we make the dataframe with these columns, we're going to divide the passLength column by the passComp column to get the number of yards players threw for on completed passes. This will be helpful for modelling.

In [81]:
nfl_total_pass_yards_completions = make_df_for_nfl_stat(nfl_passer, 'passLength', 'passAtt', 'passComp', 'passTd', 'passInt')

# make the yards per completion column
nfl_total_pass_yards_completions['NFL Yards per Completion'] = nfl_total_pass_yards_completions['passLength']/nfl_total_pass_yards_completions['passComp']
nfl_total_pass_yards_completions

,playerId,passLength,passAtt,passComp,passTd,passInt,NFL Yards per Completion
0,19850285,0,84,48,3,1,0.000000
1,19870001,1401,848,498,25,35,2.813253
2,19870098,0,118,70,4,2,0.000000
3,19880076,0,106,64,4,10,0.000000
4,19880400,-7,2,0,0,0,-inf
...,...,...,...,...,...,...,...
750,20190934,1744,216,123,7,10,14.178862
751,20190961,161,17,8,0,2,20.125000
752,20191006,5,1,0,0,0,inf
753,20191036,136,22,12,1,1,11.333333


So now we have a dataframe that tells us the total yards players have thrown for on passes, their yards thrown per completed pass, and the total passes they have thrown for both interceptions and touchdowns. Some of the players have a value of 0 for total yards thrown for because they have no data recorded for pass length for their passes. We'll keep those rows in here for now.

Let's make a dataframe for the receiver statistics. We'll keep the rec (reception) column and the recYards column, which tell us whether a play resulted in a reception and how many yards the reception resulted in, respectively. We'll also keep the recFumble column, which tells us whether a player fumbled the ball on the reception.

In [54]:
nfl_total_rec_yards = make_df_for_nfl_stat(nfl_receiver, 'rec', 'recYards', 'recFumble')
nfl_total_rec_yards

,playerId,rec,recYards,recFumble
0,19850016,36,483,0
1,19880006,28,231,1
2,19890005,0,0,0
3,19900017,17,119,0
4,19900058,75,1132,0
...,...,...,...,...
3189,20191175,6,36,0
3190,20191176,6,40,0
3191,20191178,2,12,0
3192,20191183,2,-14,0


The dataframe above has data about how many receptions players have, the total number of yards they've ran for after receptions, and the total number of fumbles on receptions.

Next we'll make a dataframe for rushing statistics (nfl_rusher dataframe). For this one we want to keep the rushYards and rushTd columns, which tell us the number of yards players rushed for on a play and whether it resulted in a touchdown.

In [55]:
nfl_total_rush_yards_tds = make_df_for_nfl_stat(nfl_rusher, 'rushYards', 'rushTd')
nfl_total_rush_yards_tds

,playerId,rushYards,rushTd
0,2504378,5,0.0
1,2505354,0,0.0
2,19850285,42,2.0
3,19870001,59,3.0
4,19870098,34,0.0
...,...,...,...
1945,20191036,50,0.0
1946,20191044,230,0.0
1947,20191077,200,0.0
1948,20191087,258,2.0


This dataframe tells us the total number of yards players rushed for and the total number of rushing touchdowns they've had, and this is represented by the rows.

Next we'll make a dataframe for the sack data in the nfl_sack dataframe. Here we'll keep the sackType column and the sackYards column. The sack type has values 1 and .5, which indicate that the sack was unassisted (1) or assisted (.5). The sackYards column tells us the total yards lost due to the sack.

In [56]:
nfl_total_sacks_yardslost = make_df_for_nfl_stat(nfl_sacks, 'sackType', 'sackYards')
nfl_total_sacks_yardslost

,playerId,sackType,sackYards
0,19890005,1.0,-5
1,19900005,8.5,-74
2,19910020,1.0,-4
3,19910025,3.0,-17
4,19910031,1.5,-13
...,...,...,...
2309,20190989,10.0,-84
2310,20190993,2.0,-10
2311,20190995,1.0,-4
2312,20191008,2.0,-16


The output above shows us we have a dataframe that tells us how many sacks (both assisted and unassisted) each player has and how many yards they've caused a loss for on their sacks. We'll change the sackType column name later on.

The last dataframe we'll change is the nfl_tackles dataframe. 

For this one, we're going to first change the tackleType column. Prior analysis indicated that this column has values of "solo", "for a loss," and "assisted." We'll change solo and for a loss tackles to 1s and assissted tackles to .5s. This will allow us to pass this column into our make_df function and figure out how many tackles players have had.

We're also going to change the name of the tackleType column to tackles so we know it gives information about total tackles. Then we'll print out the dataframe.

In [83]:
# replace values of solo and for a loss with 1
nfl_tackles = nfl_tackles.replace(to_replace = ['solo', 'for a loss'], value=1)

# replace assist with values of .5
nfl_tackles = nfl_tackles.replace(to_replace = 'assist', value=.5)

# call the function to make a new df
nfl_total_tackles = make_df_for_nfl_stat(nfl_tackles, 'tackleType')

# change name of column and print df
nfl_total_tackles = nfl_total_tackles.rename({'tackleType':'NFL Tackles'}, axis=1)
nfl_total_tackles

,playerId,NFL Tackles
0,19820086,0.5
1,19850016,2.0
2,19860201,2.0
3,19880068,1.0
4,19880400,2.5
...,...,...
7829,20191186,2.0
7830,20191189,18.0
7831,20191195,6.0
7832,20191198,2.0


The dataframe above has the total number of tackles for all players in all games. Each row represents a player

Now we'll merge all of the dataframes we just made so that we have one dataframe with total nfl statistics for players. They all contain a playerID column that remains consistent across them, so we can merge on that value.

In [85]:
all_nfl_stats = pd.merge(nfl_total_forced_fumbles, nfl_total_interceptions_yards, on='playerId', how='outer')
all_nfl_stats = pd.merge(all_nfl_stats, nfl_total_pass_yards_completions, on='playerId', how='outer')
all_nfl_stats = pd.merge(all_nfl_stats, nfl_total_rec_yards, on='playerId', how='outer')
all_nfl_stats = pd.merge(all_nfl_stats, nfl_total_rush_yards_tds, on='playerId', how='outer')
all_nfl_stats = pd.merge(all_nfl_stats, nfl_total_sacks_yardslost, on='playerId', how='outer')
all_nfl_stats = pd.merge(all_nfl_stats, nfl_total_tackles, on='playerId', how='outer')
all_nfl_stats
all_nfl_stats = all_nfl_stats.fillna(0) # fill in nans with 0s
all_nfl_stats

,playerId,NFL Forced Fumbles,fumTurnover,int,intYards,passLength,passAtt,passComp,passTd,passInt,NFL Yards per Completion,rec,recYards,recFumble,rushYards,rushTd,sackType,sackYards,NFL Tackles
0,19850285,2.0,2.0,0.0,0.0,0.0,84.0,48.0,3.0,1.0,0.000000,0.0,0.0,0.0,42.0,2.0,0.0,0.0,0.0
1,19870001,13.0,12.0,0.0,0.0,1401.0,848.0,498.0,25.0,35.0,2.813253,0.0,0.0,0.0,59.0,3.0,0.0,0.0,0.0
2,19870098,3.0,1.0,0.0,0.0,0.0,118.0,70.0,4.0,2.0,0.000000,0.0,0.0,0.0,34.0,0.0,0.0,0.0,0.0
3,19880006,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,28.0,231.0,1.0,0.0,0.0,0.0,0.0,0.0
4,19880076,1.0,1.0,0.0,0.0,0.0,106.0,64.0,4.0,10.0,0.000000,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9496,20191186,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
9497,20191189,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0
9498,20191195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
9499,20191198,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0


When we printed out the df there were a lot of nans, so we filled them with 0s. Now we have a dataframe with total statistics for all players across their NFL careers. Each row represents a player and players are identified with their playerId. 

We're ready to make one large dataframe with player information and their college and NFL statistics. Both the college and NFL statistics dataframes have an identifier for the players that match the identifiers in the player information dataframe (playerId and Player Code), so we can use these identifiers to merge them.

We'll merge the NFL statistics dataframe with the college and NFL merged player info dataframe. We'll then merge that with the college statistics dataframe. This will give us one dataframe with the NFL and college statistics for all overlapping players.

In [86]:
# merge player info df with nfl stats using the playerId column
merged_nfl_college_stats = pd.merge(merged_college_nfl, all_nfl_stats, on='playerId', how='inner')

# merge the new large df with the college stats df using player code
merged_nfl_college_stats = pd.merge(merged_nfl_college_stats, new_college_stats, on='Player Code', how='inner')

# drop any rows missing a player identifier
merged_nfl_college_stats = merged_nfl_college_stats[merged_nfl_college_stats['playerId'].notna()]

# print out info about the dataframe
merged_nfl_college_stats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 615 entries, 0 to 614
Data columns (total 49 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   playerId                        615 non-null    int64  
 1   nameFirst                       615 non-null    object 
 2   nameLast                        615 non-null    object 
 3   nameFull                        615 non-null    object 
 4   position                        615 non-null    object 
 5   collegeId                       615 non-null    int64  
 6   college                         615 non-null    object 
 7   Games played                    615 non-null    int64  
 8   nflId                           615 non-null    object 
 9   Player Code                     615 non-null    int64  
 10  Team Code                       615 non-null    int64  
 11  Position                        615 non-null    object 
 12  College Name                    615 

Now we have one large dataframe with player information and their college and NL statistics. We dropped all nan values to make sure that only rows with an identifier are included.

We're going to drop some columns that we don't need and represent duplicate information then print out our dataframe.

In [87]:
merged_nfl_college_stats = merged_nfl_college_stats.drop(['nameFull', 'college', 'College Name', 'Position'], axis=1)
merged_nfl_college_stats

,playerId,nameFirst,nameLast,position,collegeId,Games played,nflId,Player Code,Team Code,Conference Code,...,NCAA Yards per Reception,NCAA Solo Tackle per Game,NCAA Assisted Tackle per Game,NCAA Tackle for Loss per Game,NCAA Yards per Tackle for Loss,NCAA Sacks per Game,NCAA Yards per Sack,NCAA Forced Fumbles per Game,NCAA Fumbles per Game,NCAA Broken up Pass per Game
0,20120112,Bobby,Massie,OT,38677,49,2533543,1024974,433,911,...,NaN,0.10,0.00,0.00,NaN,0.00,NaN,0.00,0.00,0.00
1,20130187,Andre,Ellington,RB,29631,35,2539217,1015945,147,821,...,8.58,0.08,0.00,0.00,NaN,0.00,NaN,0.00,0.18,0.00
2,20130763,Jaron,Brown,WR,29631,56,2541966,1015966,147,821,...,14.01,0.26,0.12,0.00,NaN,0.00,NaN,0.00,0.05,0.00
3,20140052,Troy,Niklas,TE,46556,30,2543628,1040904,513,99001,...,15.49,0.32,0.43,0.02,6.00,0.00,NaN,0.00,0.00,0.00
4,20140120,Logan,Thomas,QB,24061,42,2543767,1026854,742,821,...,5.50,0.09,0.02,0.00,NaN,0.00,NaN,0.00,0.53,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
610,20170946,River,Cracraft,WR,99164,5,2559485,1058274,754,905,...,13.35,0.08,0.00,0.00,NaN,0.00,NaN,0.00,0.08,0.00
611,20170145,Jake,Butt,TE,48109,1,2557859,1056118,418,827,...,11.75,0.11,0.00,0.00,NaN,0.00,NaN,0.00,0.00,0.00
612,20180922,A.J.,Johnson,LB,37998,20,2561704,1041674,694,911,...,NaN,4.44,4.56,0.60,2.28,0.03,10.00,0.03,0.00,0.11
613,20150156,Tony,Lippett,WR,48824,4,2552427,1032668,416,827,...,12.49,0.30,0.27,0.02,2.00,0.00,NaN,0.00,0.03,0.15


Our dataframe has only columns we need.

Let's rename the columns so that we can tell which columns are NFL stats and which are college stats.

In [88]:
merged_nfl_college_stats = merged_nfl_college_stats.rename({'fumTurnover':'NFL Fumble Turnover', 'int':'NFL Interceptions', 'intYards': 'NFL Interception Yards', 'passLength': 'NFL Passing Yards', 'passTd': 'NFL Passing TDs', 'passInt': 'NFL Pass Interceptions', 'rec':'NFL Receptions', 'recYards': 'NFL Reception Yards', 'recFumble': 'NFL Fumble on Reception', 'rushYards': 'NFL Rushing Yards', 'rushTd': 'NFL Rushing TDs', 'sackType': 'NFL Sacks', 'sackYards': 'NFL Sack Yards'}, axis=1)
merged_nfl_college_stats

,playerId,nameFirst,nameLast,position,collegeId,Games played,nflId,Player Code,Team Code,Conference Code,...,NCAA Yards per Reception,NCAA Solo Tackle per Game,NCAA Assisted Tackle per Game,NCAA Tackle for Loss per Game,NCAA Yards per Tackle for Loss,NCAA Sacks per Game,NCAA Yards per Sack,NCAA Forced Fumbles per Game,NCAA Fumbles per Game,NCAA Broken up Pass per Game
0,20120112,Bobby,Massie,OT,38677,49,2533543,1024974,433,911,...,NaN,0.10,0.00,0.00,NaN,0.00,NaN,0.00,0.00,0.00
1,20130187,Andre,Ellington,RB,29631,35,2539217,1015945,147,821,...,8.58,0.08,0.00,0.00,NaN,0.00,NaN,0.00,0.18,0.00
2,20130763,Jaron,Brown,WR,29631,56,2541966,1015966,147,821,...,14.01,0.26,0.12,0.00,NaN,0.00,NaN,0.00,0.05,0.00
3,20140052,Troy,Niklas,TE,46556,30,2543628,1040904,513,99001,...,15.49,0.32,0.43,0.02,6.00,0.00,NaN,0.00,0.00,0.00
4,20140120,Logan,Thomas,QB,24061,42,2543767,1026854,742,821,...,5.50,0.09,0.02,0.00,NaN,0.00,NaN,0.00,0.53,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
610,20170946,River,Cracraft,WR,99164,5,2559485,1058274,754,905,...,13.35,0.08,0.00,0.00,NaN,0.00,NaN,0.00,0.08,0.00
611,20170145,Jake,Butt,TE,48109,1,2557859,1056118,418,827,...,11.75,0.11,0.00,0.00,NaN,0.00,NaN,0.00,0.00,0.00
612,20180922,A.J.,Johnson,LB,37998,20,2561704,1041674,694,911,...,NaN,4.44,4.56,0.60,2.28,0.03,10.00,0.03,0.00,0.11
613,20150156,Tony,Lippett,WR,48824,4,2552427,1032668,416,827,...,12.49,0.30,0.27,0.02,2.00,0.00,NaN,0.00,0.03,0.15


We've changed the column names for NFL stats so we can tell the difference between college and NFL stats.

Some of the rows have nan values in the columns, and that could create errors when we make models so we'll fill them with 0s.

In [82]:
merged_nfl_college_stats = merged_nfl_college_stats.fillna(0)
merged_nfl_college_stats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 615 entries, 0 to 614
Data columns (total 45 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   playerId                        615 non-null    int64  
 1   nameFirst                       615 non-null    object 
 2   nameLast                        615 non-null    object 
 3   position                        615 non-null    object 
 4   collegeId                       615 non-null    int64  
 5   Games played                    615 non-null    int64  
 6   nflId                           615 non-null    object 
 7   Player Code                     615 non-null    int64  
 8   Team Code                       615 non-null    int64  
 9   Conference Code                 615 non-null    int64  
 10  NFL Forced Fumbles              615 non-null    float64
 11  NFL Fumble Turnover             615 non-null    float64
 12  NFL Interceptions               615 

So now all of the columns and rows have no nan values.

We've done all the necessary data preparation work to make a dataframe suitable for the model.

Let's export this dataframe as a csv file so we can load it into a new notebook for the modelling.

In [89]:
merged_nfl_college_stats.to_csv('nfl_ncaa_stats1.csv')

Now we have a csv file with all of the data we'll need to make our model.